This is a live notebook with experimental code to develop analysis investigating the correlation between features.

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import itertools
import json

import scipy.stats as spstats
# fourier transform
from scipy.fft import fft, ifft

from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# from statsmodels.tsa.api import acf, graphics, pacf
from statsmodels.tsa.ar_model import AutoReg
# from statsmodels.tsa.ar_model import ar_select_order

import os
print(os.listdir("."))


import re
res_digit = r'[0-9]'


['debug_windowing_data.ipynb', 'testing_mlpwrapper.ipynb', 'logs', 'analyse_results.ipynb', 'gait_data_features.ipynb', 'feature_development.ipynb', 'data', 'feature_analysis.ipynb', 'test_sample_entropy_implementations.ipynb', 'merging_dataframes.ipynb', 'data_and_processing_description.ipynb', 'featured_prediction_random_forest.ipynb', 'window_timings_and_IBI.ipynb', 'gait_data_exploration.ipynb', 'processing_status.ipynb', '.ipynb_checkpoints', 'loading_e4_data.ipynb', 'data_investigation_scratch.ipynb', 'simply_load_and_inspect_data.ipynb', 'grouping_and_crossvalidation.ipynb', 'sample-entropy-numba-impl.ipynb', 'featured_prediction.ipynb']


In [2]:
# This is a hack to make the library in the parent folder available for imoprts
# A better solution is by np8 here:
# https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder
import sys
import os
import inspect

thisdir = sys.path[0]
print(f"thisdir = {thisdir}")
parentdir = os.path.dirname(thisdir)
#print(f"parentdir = {parentdir}")
if not parentdir in sys.path:
    print("Adding parent directory to python path")
    sys.path.insert(1, parentdir)
else:
    print("Skipping adding parent direct to path (there already)")

print(f"sys.path =\n{sys.path}")



thisdir = /home/luke/git/external/predicament/notebooks
Adding parent directory to python path
sys.path =
['/home/luke/git/external/predicament/notebooks', '/home/luke/git/external/predicament', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/luke/.local/lib/python3.10/site-packages', '/usr/local/lib/python3.10/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3.10/dist-packages']


In [3]:
## ensure relative path to data directory is sound
# for the notebook we need to modify the BASE_DATA_FOLDER
import os 
os.environ['PREDICAMENT_DATA_DIR'] =  '../data'

In [4]:
from predicament.utils.config import DREEM_EEG_CHANNELS
from predicament.utils.config import FEATURED_BASE_PATH
from predicament.utils.config import WINDOWED_BASE_PATH
from predicament.utils.file_utils import drop_nan_cols
from predicament.utils.file_utils import load_windowed_dataframe_and_config
from predicament.utils.file_utils import load_dataframe_and_config
from predicament.utils.file_utils import write_dataframe_and_config

from predicament.data.timeseries import create_participant_data_edf_only
from predicament.data.windowed import window_all_participants_data
from predicament.data.windowed import merge_condition_data
from predicament.data.partitioning import between_subject_cv_partition

from predicament.data.features import IDEAL_FEATURE_GROUP
from predicament.data.features import STATS_FEATURE_GROUP
from predicament.data.features import INFO_FEATURE_GROUP
from predicament.data.features import FREQ_FEATURE_GROUP
from predicament.data.features import convert_timeseries_to_features
from predicament.data.features import filter_features

from prepare_evaluation_data import group_conditions

In [5]:
from predicament.utils.config import E4_CSV_FILES
from predicament.utils.config import E4_FULL_DIRPATHS
E4_FULL_DIRPATHS

{'VG_01': '../data/CARE_HOME_DATA/./VG01/E4_8921_15_44/',
 'VG_03': '../data/CARE_HOME_DATA/./VG03/E4_9921_12_16/',
 'VG_05': '../data/CARE_HOME_DATA/./VG05/E4_9921_13_24/',
 'VG_06': '../data/CARE_HOME_DATA/./VG06/E4_51021_13_33/',
 'VG_07': '../data/CARE_HOME_DATA/./VG07/E4_51021_15_39/',
 'VG_08': '../data/CARE_HOME_DATA/./VG08/E4_71021_10_42/',
 'VG_09': '../data/CARE_HOME_DATA/./VG09/E4_11221_14_46/',
 'VG_10': '../data/CARE_HOME_DATA/./VG10/E4_31221_11_17/',
 'VH_01': '../data/CARE_HOME_DATA/./VH01/E4_61021_11_03/',
 'VH_02': '../data/CARE_HOME_DATA/./VH02/E4_61021_13_59/',
 'VH_03': '../data/CARE_HOME_DATA/./VH03/E4_11221_11_22/'}

In [6]:
subdir = 'binary_dreem_4secs'
df, config = load_windowed_dataframe_and_config(subdir)
df

KeyboardInterrupt: 

In [ ]:
config

In [ ]:
df['condition'].unique()



In [ ]:
string_ = "inactive:baseline,break;active:exper_video,wildlife_video,familiar_music,tchaikovsky,family_inter"
dict_ = get_dict_from_string(string_)
dict_

In [ ]:
get_dict_of_lists_from_dict_of_strings(dict_)

In [7]:
IDEAL_FEATURE_GROUP

{'Correlation',
 'FreqKurtosis',
 'HRVRMSSD',
 'Hurst',
 'IQR',
 'LempelZivComplexity',
 'LyapunovExponent',
 'MAD',
 'Max',
 'MaxFreqInd',
 'Mean',
 'MeanFreq',
 'Min',
 'SD',
 'SampleEntropy',
 'arCoeff'}

In [13]:
feature_set  = INFO_FEATURE_GROUP 
#feature_set &= IDEAL_FEATURE_GROUP
feature_set

{'Hurst',
 'LempelZivComplexity',
 'LyapunovExponent',
 'SampleEntropy',
 'arCoeff'}

In [14]:
existing_df = pd.read_csv('/tmp/existing_df.csv')
new_df = pd.read_csv('/tmp/new_df.csv')
label_cols = ['participant', 'condition', 'window index']

In [15]:
from predicament.data.features import add_features_to_dataframe

In [16]:
add_features_to_dataframe(
        existing_df, new_df, label_cols)

ValueError: Cannot merge as rows do not coincide

In [17]:
merge_on = label_cols
left_cols = existing_df.columns
right_cols = new_df.columns
# new dataframe will overwrite any columns that are not labels and have
# matching column names
preserved_left_cols = [
    c for c in left_cols if (c in merge_on) or (not c in right_cols) ]
# hide the columns that will be overridden
existing_df = existing_df.loc[:,preserved_left_cols]
print(f"len(existing_df.index) = {len(existing_df.index)}")
print(f"len(new_df.index) = {len(new_df.index)}")
print(f"merge_on = {merge_on}")
for col in merge_on:
    print(f"existing_df[col].dtype = {existing_df[col].dtype}")    
    print(f"new_df[col].dtype = {new_df[col].dtype}")    
    print(f"existing_df[col].unique() = {existing_df[col].unique()}")    
    print(f"new_df[col].unique() = {new_df[col].unique()}")    
result_df = pd.merge(existing_df, new_df, how='inner', on=merge_on)
print(f"len(result_df.index) = {len(result_df.index)}")
if len(existing_df.index) != len(result_df.index):
    existing_df.to_csv('/tmp/existing_df.csv')
    new_df.to_csv('/tmp/new_df.csv')
    raise ValueError('Cannot merge as rows do not coincide')

len(existing_df.index) = 29698
len(new_df.index) = 29698
merge_on = ['participant', 'condition', 'window index']
existing_df[col].dtype = object
new_df[col].dtype = object
existing_df[col].unique() = ['VG_01' 'VG_03' 'VG_05' 'VG_06' 'VG_07' 'VG_08' 'VG_09' 'VG_10' 'VH_01'
 'VH_02' 'VH_03']
new_df[col].unique() = ['VG_01' 'VG_03' 'VG_05' 'VG_06' 'VG_07' 'VG_08' 'VG_09' 'VG_10' 'VH_01'
 'VH_02' 'VH_03']
existing_df[col].dtype = int64
new_df[col].dtype = int64
existing_df[col].unique() = [0 1]
new_df[col].unique() = [0 1]
existing_df[col].dtype = int64
new_df[col].dtype = int64
existing_df[col].unique() = [   0    1    2 ... 1070 1071 1072]
new_df[col].unique() = [   0    1    2 ... 1070 1071 1072]
len(result_df.index) = 99104


ValueError: Cannot merge as rows do not coincide